# w261 Final Project - Clickthrough Rate Prediction


[Your team number (from the spreadsheet)]   
[Your team names]   
Summer 2019, section [Your section numbers>]   

## Table of Contents

* __Section 1__ - Question Formulation
* __Section 2__ - Algorithm Explanation
* __Section 3__ - EDA & Challenges
* __Section 4__ - Algorithm Implementation
* __Section 5__ - Course Concepts

# __Section 1__ - Question Formulation

# __Section 2__ - Algorithm Explanation

How do we justify the selection of a particular algorithm for our project?
1. when data is sparse, FM and FFM work better (need to see if the data is sparse)
2. how to justify the choice of FFM over neronets, and trees and other algorithm??

# __Section 3__ - EDA & Challenges

Do we run EDA on the entire dataset or just the sample train data?

In [88]:
from pyspark.sql import types
from pyspark.sql import SQLContext
from pyspark.sql.functions import isnan
from pyspark.sql import functions as F

In [2]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [4]:
# read training data from bucket and write the parquet format of the file to bucket
# train_data = spark.read.csv("gs://w261-bucket-hou/final-project/data/train.txt", sep="\t")
# train_data.write.format("parquet").save("gs://w261-bucket-hou/final-project/data/train.parquet")

In [82]:
#read the parquet file
train_parquet = spark.read.parquet("gs://w261-bucket-hou/final-project/data/train.parquet")
oldColNames = train_parquet.schema.names

In [86]:
!gsutil du -sh gs://w261-bucket-hou/final-project/data/train.txt
!gsutil du -sh gs://w261-bucket-hou/final-project/data/train.parquet

10.38 GiB   gs://w261-bucket-hou/final-project/data/train.txt
2.74 GiB    gs://w261-bucket-hou/final-project/data/train.parquet


In [77]:
train_parquet.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [83]:
#rename files and recast integer types on the first 13 features
train_parquet = train_parquet.withColumn("label", train_parquet["_c0"])
for colNum in range(1,14): 
    colName = "_c" + str(colNum)
    train_parquet = train_parquet.withColumn("int_feature_"+ str(colNum), train_parquet[colName].cast(types.IntegerType()))
for colNum in range(14,40): 
    colName = "_c" + str(colNum)
    train_parquet = train_parquet.withColumn("cate_feature_"+ str(colNum-13), train_parquet[colName])
    
train_parquet = train_parquet.drop(*oldColNames)

In [84]:
train_parquet.printSchema()

root
 |-- label: string (nullable = true)
 |-- int_feature_1: integer (nullable = true)
 |-- int_feature_2: integer (nullable = true)
 |-- int_feature_3: integer (nullable = true)
 |-- int_feature_4: integer (nullable = true)
 |-- int_feature_5: integer (nullable = true)
 |-- int_feature_6: integer (nullable = true)
 |-- int_feature_7: integer (nullable = true)
 |-- int_feature_8: integer (nullable = true)
 |-- int_feature_9: integer (nullable = true)
 |-- int_feature_10: integer (nullable = true)
 |-- int_feature_11: integer (nullable = true)
 |-- int_feature_12: integer (nullable = true)
 |-- int_feature_13: integer (nullable = true)
 |-- cate_feature_1: string (nullable = true)
 |-- cate_feature_2: string (nullable = true)
 |-- cate_feature_3: string (nullable = true)
 |-- cate_feature_4: string (nullable = true)
 |-- cate_feature_5: string (nullable = true)
 |-- cate_feature_6: string (nullable = true)
 |-- cate_feature_7: string (nullable = true)
 |-- cate_feature_8: string (nulla

In [85]:
#record feature names by feature type
# intFieldNames = [colName for colName, dType in train_parquet.dtypes if dType == 'int']
# cateFieldNames = [colName for colName, dType in train_parquet.dtypes if dType == 'string' and colName != 'label']

In [93]:
recCount = train_parquet.count() #get record count
fieldCount = len(train_parquet.columns)-1 #get the count of features

In [89]:
distinctValues = []
nullValues = []
for colName in train_parquet.schema.names:
    distinctValues.append(train_parquet.select(colName).distinct().count())
    nullValues.append(train_parquet.filter((train_parquet[colName] == "") | train_parquet[colName].isNull() | isnan(train_parquet[colName])).count())

In [109]:
print("Training Dataset Overview:\n")
print("Record Count:", recCount)
print("Field Count:", fieldCount)
print("Null Value Percentage:", str(round(float(sum(nullValues))/(recCount*fieldCount)*100,1)) + "%")

print("="*70, "\n")


print("Column Level Information:\n")
print('%-15s| %-15s| %-15s| %-15s' %('Field Name','Distinct Count', 'Null Count', 'Null Percent'))
print('-'*70)
for colName, distinctVal, nullVal in zip(train_parquet.schema.names, distinctValues, nullValues):
    print('%-15s| %-15s| %-15s| %-15s' %(colName, distinctVal, nullVal, str(round(float(nullVal)/recCount*100,1)) + "%" ))

Training Dataset Overview:

Record Count: 45840617
Field Count: 39
Null Value Percentage: 14.2%

Column Level Information:

Field Name     | Distinct Count | Null Count     | Null Percent   
----------------------------------------------------------------------
label          | 2              | 0              | 0.0%           
int_feature_1  | 649            | 20793556       | 45.4%          
int_feature_2  | 9364           | 0              | 0.0%           
int_feature_3  | 14746          | 9839447        | 21.5%          
int_feature_4  | 490            | 9937369        | 21.7%          
int_feature_5  | 476707         | 1183117        | 2.6%           
int_feature_6  | 11618          | 10252328       | 22.4%          
int_feature_7  | 4142           | 1982866        | 4.3%           
int_feature_8  | 1373           | 22773          | 0.0%           
int_feature_9  | 7275           | 1982866        | 4.3%           
int_feature_10 | 13             | 20793556       | 45.4%          
i

Callenges will be caused by dataset being very sparse:
1. null values in the training data
2. the zeros created by one-hot encoding of the categorical variables (too many distinct values for the categorical variables)

Solutions:
1. feature selections
2. hashing categorical variables

# __Section 4__ - Algorithm Implementation

# __Section 5__ - Course Concepts